In [342]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
from dfply import *
import numpy as np

In [150]:
df = pd.read_csv('../Data/df_satisfaction_rdq_dx_demo.csv', low_memory=False)


In [152]:
dfMDD = df[df.mddsnp_1.str.contains('Curr, Prin') | df.mddrnp_1.str.contains('Curr, Prin')]

In [183]:
dfMDD.loc[:,('pre_dsym')] = dfMDD.filter(regex='rdqpre_[1-9]_|rdqpre_1[0-4]_').mean(axis = 1)
dfMDD['pre_ndsym'] = dfMDD.filter(regex='rdqpre_1[5-9]_|rdqpre_2[0-5]_').mean(axis = 1)
dfMDD['post_dsym'] = dfMDD.filter(regex='rdqpost_[1-9]_|rdqpost_1[0-4]_').mean(axis = 1)
dfMDD['post_ndsym'] = dfMDD.filter(regex='rdqpost_1[5-9]_|rdqpost_2[0-5]_').mean(axis = 1)

In [185]:
labels = ['dsym', 'ndsym', 'cope', 'pmh', 'fun', 'well']
for label in labels:
    dfMDD['change_'+ label] = dfMDD.filter(regex='^post_'+label).squeeze() - dfMDD.filter(regex='^pre_'+label).squeeze()


dfMDD.change_dsym = dfMDD.change_dsym*(-1)
dfMDD.change_ndsym = dfMDD.change_ndsym*(-1)
dfMDD.to_csv('../Data/dfMDD_satisfaction_rdq_dx_demo.csv')

In [186]:
print(
    'Number of People with Signifiacnt Depressive Symptom Reduction:', sum(dfMDD.change_dsym >= 0.5), '\n',
    'Number of People with Signifiacnt Non-Depressive Symptom Reduction:', sum(dfMDD.change_ndsym >= 0.5), '\n',
    'Number of People with Signifiacnt Coping Improvement:', sum(dfMDD.change_cope >= 0.5), '\n',
    'Number of People with Signifiacnt Positve Mental Health Improvement:', sum(dfMDD.change_pmh >= 0.5), '\n',
    'Number of People with Signifiacnt Functioning Improvement:', sum(dfMDD.change_fun >= 0.5), '\n',
    'Number of People with Signifiacnt Well-Being Improvement:', sum(dfMDD.change_well >= 0.5))

Number of People with Signifiacnt Depressive Symptom Reduction: 672 
 Number of People with Signifiacnt Non-Depressive Symptom Reduction: 628 
 Number of People with Signifiacnt Coping Improvement: 607 
 Number of People with Signifiacnt Positve Mental Health Improvement: 668 
 Number of People with Signifiacnt Functioning Improvement: 569 
 Number of People with Signifiacnt Well-Being Improvement: 715


In [197]:
cut = 0.5
for label in labels:
    dfMDD['change_'+label+'_cut'] = dfMDD['change_'+label] >= 0.5
    dfMDD['change_'+label+'_cut'] = dfMDD['change_'+label+'_cut'].replace({True: 1, False: 0}).astype('category')

In [373]:
contingencyTable = pd.crosstab(dfMDD['change_dsym_cut'], dfMDD['change_'+label+'_cut'])
percentageTable = pd.crosstab(dfMDD['change_dsym_cut'], dfMDD['change_'+label+'_cut'], normalize='all')
    
for label in labels[2:]:
    contingencyTable = pd.concat([contingencyTable, pd.crosstab(dfMDD['change_dsym_cut'], 
                                                                dfMDD['change_'+label+'_cut'])], axis = 1)
    percentageTable = pd.concat([percentageTable, pd.crosstab(dfMDD['change_dsym_cut'], 
                                                                dfMDD['change_'+label+'_cut'],
                                                              normalize='all')], axis = 1)


In [374]:
category = [0,1]*5
contingencylabel = np.repeat(labels[1:],2)
namelist = []
for i in range(10):
    namelist.append(contingencylabel[i] +' '+str(category[i]) )
    
contingencyTable.columns = namelist
percentageTable.columns = namelist  

In [375]:
contingencyTable

,ndsym 0,ndsym 1,cope 0,cope 1,pmh 0,pmh 1,fun 0,fun 1,well 0,well 1
change_dsym_cut,,,,,,,,,,
0,355,176,378,153,388,143,398,133,355,176
1,133,539,218,454,147,525,236,436,133,539


In [376]:
percentageTable

,ndsym 0,ndsym 1,cope 0,cope 1,pmh 0,pmh 1,fun 0,fun 1,well 0,well 1
change_dsym_cut,,,,,,,,,,
0,0.295096,0.146301,0.314214,0.127182,0.322527,0.118869,0.330840,0.110557,0.295096,0.146301
1,0.110557,0.448047,0.181214,0.377390,0.122195,0.436409,0.196176,0.362427,0.110557,0.448047


In [290]:
dfMDD.filter(regex= 'change_('+'|'.join(map(str, labels)) +')_cut').value_counts().to_markdown()

'|                    |   0 |\n|:-------------------|----:|\n| (1, 1, 1, 1, 1, 1) | 257 |\n| (0, 0, 0, 0, 0, 0) | 199 |\n| (1, 1, 0, 1, 1, 1) |  51 |\n| (1, 1, 1, 1, 0, 1) |  47 |\n| (1, 0, 1, 1, 1, 1) |  47 |\n| (0, 1, 0, 0, 0, 0) |  34 |\n| (0, 0, 1, 0, 0, 0) |  33 |\n| (1, 1, 0, 1, 0, 1) |  30 |\n| (0, 0, 0, 0, 0, 1) |  29 |\n| (1, 1, 0, 0, 0, 0) |  26 |\n| (0, 0, 0, 0, 1, 0) |  25 |\n| (0, 0, 1, 1, 1, 1) |  25 |\n| (0, 0, 0, 1, 0, 0) |  20 |\n| (0, 1, 1, 1, 1, 1) |  17 |\n| (1, 0, 1, 1, 0, 1) |  16 |\n| (1, 0, 0, 0, 0, 0) |  16 |\n| (0, 0, 0, 1, 0, 1) |  16 |\n| (1, 1, 0, 0, 0, 1) |  15 |\n| (1, 1, 1, 0, 0, 0) |  15 |\n| (1, 0, 0, 1, 1, 1) |  13 |\n| (0, 0, 0, 1, 1, 1) |  12 |\n| (0, 0, 1, 0, 0, 1) |  12 |\n| (0, 0, 1, 1, 0, 1) |  12 |\n| (1, 1, 1, 1, 1, 0) |  11 |\n| (1, 0, 0, 1, 0, 1) |  11 |\n| (1, 1, 1, 0, 1, 1) |  11 |\n| (1, 1, 1, 0, 0, 1) |  11 |\n| (0, 0, 0, 0, 1, 1) |  10 |\n| (1, 1, 0, 1, 0, 0) |  10 |\n| (1, 0, 1, 0, 0, 0) |   9 |\n| (1, 0, 1, 0, 1, 1) |   9 |\n| (1, 0, 

In [288]:
dfMDD.change_dsym_cut

6       1
7       1
14      0
15      0
17      0
       ..
2679    0
2680    1
2682    0
2683    0
2684    0
Name: change_dsym_cut, Length: 1203, dtype: category
Categories (2, int64): [0, 1]

In [250]:
'change_['+' +']_cut'


|'.join(map(str, labels[1:]))

'change_[ndsym|cope|pmh|fun|well]_cut'